In [1]:
from seligator.main import train_and_get, Seligator
from seligator.common.load_save import load
from seligator.prediction.tests import run_tests

params = load("./seligator/siamese.json")
params["reader_kwargs"]["batch_size"] = 16
params["reader_kwargs"]["dev_batch_size"] = 32
#params["batches_per_epoch"] = 5


2021-08-16 17:47:24.005985: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
seligator, reader, train, dev = Seligator.init_from_params(**params)
train_kwargs = dict(patience=4, num_epochs=50, lr=5e-4, optimizer="AdamW")
print(seligator.model)

/home/thibault/dev/est-lascivuum-non-est/env/lib/python3.8/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:55: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  warnings.warn(


building vocab:   0%|          | 0/21955 [00:00<?, ?it/s]

SiameseClassifier(
  (_loss): ContrastiveLoss(
    (distance): LpDistance()
    (reducer): MultipleReducers(
      (reducers): ModuleDict(
        (pos_loss): AvgNonZeroReducer()
        (neg_loss): AvgNonZeroReducer()
      )
      (default_reducer): MeanReducer()
    )
  )
  (left_encoder): MixedEmbeddingEncoder(
    (_emb_dropout): Dropout(p=0.3, inplace=False)
    (features_embedder): FeatureAndTextEmbedder(
      (text_embedder): BasicTextFieldEmbedder(
        (token_embedder_token): Embedding()
        (token_embedder_lemma): Embedding()
        (token_embedder_lemma_char): TokenCharactersEncoder(
          (_embedding): TimeDistributed(
            (_module): Embedding()
          )
          (_encoder): TimeDistributed(
            (_module): LstmSeq2VecEncoder(
              (_module): LSTM(100, 150, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
            )
          )
          (_dropout): Dropout(p=0.3, inplace=False)
        )
      )
      (feature_em

/home/thibault/dev/est-lascivuum-non-est/env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [3]:
model = train_and_get(seligator.model, train, dev, **train_kwargs)
data, img = run_tests(
    "dataset/main/test.txt",
    dataset_reader=reader, model=model, dump=f"siamese.best.csv"
)
seligator.save_model("siamese_saved_model")

---> Epochs:   50
---> Patience: 4


  0%|          | 0/1373 [00:00<?, ?it/s]

torch.Size([16, 50])
tensor([[ 0.0000, -0.4233,  0.2099,  0.4136,  0.0000,  0.3802,  0.0000, -0.0177,
         -0.0034,  0.0000,  0.0000, -0.0234,  0.0000,  0.1040,  0.0000,  0.0000,
          0.1698,  0.0619,  0.0000,  0.0000,  0.0768, -0.1395,  0.0000,  0.0389,
          0.0000,  0.0000,  0.0000, -0.2538, -0.8100, -0.6300,  0.0000, -0.3703,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0371,  0.0000,  0.0000,
         -0.3589,  0.0000,  0.0000, -0.1459, -0.4604,  0.0000,  0.0000, -0.2193,
          0.0000,  0.0000],
        [ 0.0841, -0.4167,  0.0000,  0.4466,  0.0000,  0.5560,  0.0000,  0.0000,
          0.0000,  0.0000,  0.2312, -0.0209, -0.2353,  0.2078, -0.0438, -0.2636,
          0.5456,  0.0000,  0.5773, -0.0066,  0.1834, -0.1682,  0.0958,  0.0000,
          0.0000,  0.0000,  0.6819, -0.1265, -0.5892,  0.0000,  0.3286,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.1904,  0.0000, -0.0462,  0.0000,
         -0.5957,  0.1885, -0.1362,  0.0000,  0.0000, -0.311

ValueError: labels must be a 1D tensor of shape (batch_size,)